In [2]:
import sys
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import math
from tensorflow.keras.callbacks import EarlyStopping

2021-10-29 15:10:02.553575: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-10-29 15:10:02.553623: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import struct

def bin2float(b):
    ''' Convert binary string to a float.

    Attributes:
        :b: Binary string to transform.
    '''
    h = int(b, 2).to_bytes(8, byteorder="big")
    return struct.unpack('>d', h)[0]


def float2bin(f):
    ''' Convert float to 64-bit binary string.

    Attributes:
        :f: Float number to transform.
    '''
    [d] = struct.unpack(">Q", struct.pack(">d", f))
    return f'{d:064b}'

In [4]:
def quantize_float (float_number, mantissa_bits):
    if mantissa_bits < 0:
      mantissa_bits = 0
    coefficient = float2bin(float_number)
    sign = int (coefficient[:1], 2)
    if 0 < mantissa_bits:
        custom_mantissa = int (coefficient[12 : 12 + mantissa_bits], 2)
    else:
        custom_mantissa = 0
    residual_mantissa = int (coefficient[12 + mantissa_bits:], 2)
    exponent = int (coefficient[1:12], 2) - 1023
    if (pow (2, (52 - (mantissa_bits + 1))) - 1) < residual_mantissa:
        custom_mantissa += 1
        if (pow (2, mantissa_bits) - 1) < custom_mantissa:
            custom_mantissa = 0
            exponent += 1
    return pow(-1, sign) * (1 + custom_mantissa * pow(2, - mantissa_bits)) * pow(2, exponent)

You can test here the quantizer

In [6]:
float_number = math.pi
mantissa_bits = 3
print ("___ Quantizer test ___")
print ("Double float value = {}, mantissa bit size = {}, quantized value = {}".format(float_number, mantissa_bits, quantize_float(float_number, mantissa_bits)))

___ Quantizer test ___
Double float value = 3.141592653589793, mantissa bit size = 3, quantized value = 3.25
